In [ ]:
!pip install arch
!pip install pytz
!pip install math
!pip install yfinance

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import pytz
import arch
import math
import yfinance as yf

## Fetch data from TradingView

In [ ]:
# Input parameters, start date, time, interval
sym = "EURUSD=X"
start = "2024-03-01"
end = "2024-03-02"
inter = "15min"

fetch_data = yf.download(sym, start, end, interval=inter)
raw = fetch_data.copy()

## Applying returns and Value At Risk simulation

In [ ]:
monte_ret = raw.copy()
monte_ret["pips"] = (monte_ret["Close"] - monte_ret["Close"].shift(1))
monte_ret = monte_ret.dropna()

# Percentage Returns
monte_ret["pct_returns"] = monte_ret["Close"].pct_change().dropna()
monte_ret = monte_ret.dropna()

# Sort returns in ascending order
monte_ret_var = monte_ret.pct_returns

# Predicting volatility at time t+1 for monte carlo simulation
df = pd.DataFrame(monte_ret, columns=['pct_returns'])

# Rescale the returns data
df['scaled_returns'] = df['pct_returns'] * 10000


# GARCH(1,1) with rolling window = 2

rolling_window = 2
forecasts_variance = []  # Use forecasts_variance list to store the predicted variances
forecasts_mean = []  # Use forecasts_mean list to store the predicted means

for i in range(len(df['scaled_returns']) - rolling_window + 1):  # Note the +1 to ensure inclusive window

    # Slice the rescaled returns for the current rolling window
    windowed_returns = df['scaled_returns'].iloc[i : i + rolling_window]

    # Fit GARCH(1,1) model for the current window
    model = arch.arch_model(windowed_returns, vol='Garch', p=1, q=1)
    results = model.fit()

    # Forecast volatility and mean for the next time step
    forecast = results.forecast(horizon=1)
    forecast_variance = np.sqrt(forecast.variance.values[-1, -1]) / 10000  # Convert back to original scale
    forecast_mean = forecast.mean.values[-1, -1] / 10000  # Convert back to original scale

    forecasts_variance.append(forecast_variance)
    forecasts_mean.append(forecast_mean)

# Monte Carlo

In [ ]:
# Number of simulations
num_simulations = 200000

# Initialize arrays to store VaR results
var_95_mean = []
var_95_variance = []

# Perform Monte Carlo simulation and calculate 95% VaR for each forecasted mean and variance
for mean, variance in zip(monte_ret_var.shift(1), forecasts_variance):
    # Generate simulated returns for the current forecasted mean and variance
    simulated_returns = np.random.normal(mean, np.sqrt(variance), num_simulations)
    
    # Calculate the 95% VaR for the simulated return series
    var_95 = np.percentile(simulated_returns, 0.05)
    
    # Append VaR results to the arrays
    var_95_mean.append(var_95)

In [ ]:
monte_ret_2 = monte_ret.copy()
monte_ret_2 = monte_ret_2.reset_index()
monte_ret_2.set_index("Datetime")

checker = monte_ret_2.copy()  # Create a copy of monte_ret_2 DataFrame
checker = checker[1:]
# Assuming var_95_mean is a Series containing the VaR values

# Account for spreads
#checker["low_spread"] = checker.Low + checker.spread/100

checker['VaR_95'] = var_95_mean  # Add a new column 'VaR_95' to the checker DataFrame

# Determine price of VaR
checker['VaR_95_price'] = ((1-abs(checker.VaR_95)/200) / (1+abs(checker.VaR_95)/200)) * checker.Low

checker["pct_returns"] = checker["pct_returns"]

checker["HIT"] = (checker["Low"] < checker["VaR_95_price"].shift(1)).astype(int)

#pd.set_option('display.max_rows', 50)
print(pd.DataFrame(checker[["Low", "VaR_95_price", "HIT"]]))

## Apply Poisson to determine the probability of HITS

In [ ]:
import mplfinance as mpf
import scipy.stats as stats
rate_parameter = checker['HIT'].mean()

# Define the number of occurrences for which you want to calculate the probability
k = 1  # Define the number of occurrences for which you want to calculate the probability

# Calculate the probability of k occurrences using the Poisson distribution
probability = stats.poisson.pmf(k, rate_parameter)
print("Probability of 1 occurrence:", probability)

## Plot graph and VaR

In [ ]:
data = monte_ret_2.copy()

# Convert index to DatetimeIndex
data.index = pd.to_datetime(data.index)

# Assuming 'pct_returns' is the column name containing the percentage returns data
pct_returns = checker['VaR_95_price']

# Plot the candlestick chart with the percentage returns as an additional plot
mpf.plot(data[1:], type='candle', style='yahoo', addplot=[
    mpf.make_addplot(pct_returns, color='green', alpha=0.5)  # Percentage returns as an additional plot
], title='{} Chart with % Returns'.format(sym))